In [1]:
"""
1. Data provider
    a. Image data 
    b. random vector
2. Build compute graph
    a. generator
    b. discriminator
    c. DCGAN
        connect g and d
        define loss
        define train_op
3. training process
"""
import os
import sys
import tensorflow as tf
from tensorflow import gfile
import pprint
import pickle
import numpy as np
import random 
import math
from PIL import Image
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

output_dir = './local_run'
if not gfile.Exists(output_dir):
    gfile.MakeDirs(output_dir)
    
def get_default_pramas():
    return tf.contrib.training.HParams(
        z_dim = 100,
        init_conv_size = 4,
        g_channels = [128, 64, 32, 1], # 生成器
        d_channels = [32, 64, 128, 256], # 判别器
        batch_size = 128,
        learning_rate = 0.002,
        beta1 = 0.5,
        img_size = 32,
    )

hps = get_default_pramas()

W0831 12:41:13.716003 140308269344576 deprecation.py:323] From <ipython-input-1-82994a59d565>:26: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0831 12:41:13.716754 140308269344576 deprecation.py:323] From /root/anaconda3/envs/deep_learning/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0831 12:41:13.717190 140308269344576 deprecation.py:323] From /root/anaconda3/envs/deep_learning/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and

Extracting MNIST_data/train-images-idx3-ubyte.gz


W0831 12:41:13.919804 140308269344576 deprecation.py:323] From /root/anaconda3/envs/deep_learning/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0831 12:41:13.921242 140308269344576 deprecation.py:323] From /root/anaconda3/envs/deep_learning/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0831 12:41:13.959895 140308269344576 deprecation.py:323] From /root/anaconda3/envs/deep_learning/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.con

Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
class MnistData(object):
    def __init__(self, mnist_train, z_dim, img_size):
        self._data = mnist_train
        self._example_num = len(self._data)
        self._z_data = np.random.standard_normal(
            (self._example_num, z_dim))
        self._indicator = 0
        self._random_shuffle()
        self._resize_mnist_img(img_size)
        self._random_shuffle()
    
    def _random_shuffle(self):
        p = np.random.permutation(self._example_num)
        self._z_data = self._z_data[p]
        self._data = self._data[p]
        
    def _resize_mnist_img(self, img_size):
        """Resize mnist image to goal img_size
        How?
        1. numpy -> PIL img
        2. PIL img -> resize
        3. PIL img -> numpy
        """
        data = np.asarray(self._data * 255, np.uint8)
        # [example_num, 784] -> [example_num, 28, 28]
        data = data.reshape((self._example_num, 28, 28))
        new_data = []
        for i in range(self._example_num):
            img = data[i]
            img = Image.fromarray(img)
            img = img.resize((img_size, img_size))
            img = np.asarray(img)
            img = img.reshape((img_size, img_size, 1))
            new_data.append(img)
        new_data = np.asarray(new_data, dtype=np.float32)
        new_data = new_data / 127.5 - 1
        # self._data: [num_example, img_size, img_size, 1]
        self._data = new_data
        
    def next_batch(self, batch_size):
        end_indicator = self._indicator + batch_size
        if end_indicator > self._example_num:
            self._random_shuffle()
            self._indicator = 0
            end_indicator = self._indicator + batch_size
        assert end_indicator < self._example_num
        
        batch_data = self._data[self._indicator: end_indicator]
        batch_z = self._z_data[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_z
    
mnist_data = MnistData(mnist.train.images, hps.z_dim, hps.img_size)
batch_data, batch_z = mnist_data.next_batch(5)

In [3]:
def conv2d_transpose(inputs, out_channel, name, training, with_bn_relu=True):
    """将生成器要使用的卷积层函数打包，增加batch normalization层"""
    with tf.variable_scope(name):
        conv2d_trans = tf.layers.conv2d_transpose(inputs, 
                                                  out_channel,
                                                  [5, 5],
                                                  strides=(2,2),
                                                  padding='SAME')
        if with_bn_relu:
            bn = tf.layers.batch_normalization(conv2d_trans, training=training)
            relu = tf.nn.relu(bn)
            return relu
        else:
            return conv2d_trans
def conv2d(inputs, out_channel, name, training):
    """将判别器要使用的卷积层函数打包，使用Leaky_relu激活函数"""
    def leaky_relu(x, leak = 0.2, name = ''):
        return tf.maximum(x, x * leak, name = name)
    with tf.variable_scope(name):
        conv2d_output = tf.layers.conv2d(inputs,
                                         out_channel,
                                         [5, 5],
                                         strides = (2, 2),
                                         padding = 'SAME')
        bn = tf.layers.batch_normalization(conv2d_output,
                                           training = training)
        return leaky_relu(bn, name = 'outputs')
    
class Generator(object):
    """生成器"""
    def __init__(self, channels, init_conv_size):
        assert len(channels) > 1
        self._channels = channels
        self._init_conv_size = init_conv_size
        self._reuse = False
    
    def __call__(self, inputs, training):
        inputs = tf.convert_to_tensor(inputs)
        with tf.variable_scope('generator', reuse=self._reuse):
            with tf.variable_scope('inputs'):
                fc = tf.layers.dense(
                    inputs, 
                    self._channels[0] * self._init_conv_size * self._init_conv_size)
                conv0 = tf.reshape(fc, [-1, self._init_conv_size, self._init_conv_size, self._channels[0]])
                bn0 = tf.layers.batch_normalization(conv0, training=training)
                relu0 = tf.nn.relu(bn0)
            
            deconv_inputs = relu0
            # deconvolutions * 4
            for i in range(1, len(self._channels)):
                with_bn_relu = (i != len(self._channels) - 1)
                deconv_inputs = conv2d_transpose(deconv_inputs,
                                                 self._channels[i],
                                                 'deconv-%d' % i,
                                                 training,
                                                 with_bn_relu)
            img_inputs = deconv_inputs
            with tf.variable_scope('generate_imgs'):
                # imgs value scope: [-1, 1]
                imgs = tf.tanh(img_inputs, name='imgaes')
        self._reuse=True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='generator')
        return imgs
class Discriminator(object):
    """判别器"""
    def __init__(self, channels):
        self._channels = channels
        self._reuse = False
    
    def __call__(self, inputs, training):
        inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
        
        conv_inputs = inputs
        with tf.variable_scope('discriminator', reuse = self._reuse):
            for i in range(len(self._channels)):
                conv_inputs = conv2d(conv_inputs,
                                     self._channels[i],
                                     'deconv-%d' % i,
                                     training)
            fc_inputs = conv_inputs
            with tf.variable_scope('fc'):
                flatten = tf.layers.flatten(fc_inputs)
                logits = tf.layers.dense(flatten, 2, name="logits")
        self._reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='discriminator')
        return logits
    
class DCGAN(object):
    """建立DCGAN模型"""
    def __init__(self, hps):
        g_channels = hps.g_channels
        d_channels = hps.d_channels
        
        self._batch_size = hps.batch_size
        self._init_conv_size = hps.init_conv_size
        self._batch_size = hps.batch_size
        self._z_dim = hps.z_dim
        
        self._img_size = hps.img_size
        
        self._generator = Generator(g_channels, self._init_conv_size)
        self._discriminator = Discriminator(d_channels)
    
    def build(self):
        self._z_placholder = tf.placeholder(tf.float32, (self._batch_size, self._z_dim))
        self._img_placeholder = tf.placeholder(tf.float32, 
                                               (self._batch_size, self._img_size, self._img_size, 1))
        
        generated_imgs = self._generator(self._z_placholder, training = True)
        fake_img_logits = self._discriminator(generated_imgs, training = True)
        real_img_logits = self._discriminator(self._img_placeholder, training = True)
        
        loss_on_fake_to_real = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels = tf.ones([self._batch_size], dtype = tf.int64),
                logits = fake_img_logits))
        loss_on_fake_to_fake = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels = tf.zeros([self._batch_size], dtype = tf.int64),
                logits = fake_img_logits))
        loss_on_real_to_real = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels = tf.ones([self._batch_size], dtype = tf.int64),
                logits = real_img_logits))
        
        tf.add_to_collection('g_losses', loss_on_fake_to_real)
        tf.add_to_collection('d_losses', loss_on_fake_to_fake)
        tf.add_to_collection('d_losses', loss_on_real_to_real)
        
        
        loss = {
            'g': tf.add_n(tf.get_collection('g_losses'), name = 'total_g_loss'),
            'd': tf.add_n(tf.get_collection('d_losses'), name = 'total_d_loss')
        }
        
        return (self._z_placholder, self._img_placeholder, generated_imgs, loss)
    
    def build_train(self, losses, learning_rate, beta1):
        g_opt = tf.train.AdamOptimizer(learning_rate = learning_rate, beta1 = beta1)
        d_opt = tf.train.AdamOptimizer(learning_rate = learning_rate, beta1 = beta1)
        g_opt_op = g_opt.minimize(losses['g'], var_list = self._generator.variables)
        d_opt_op = d_opt.minimize(losses['d'], var_list = self._discriminator.variables)
        with tf.control_dependencies([g_opt_op, d_opt_op]):
            return tf.no_op(name = 'train')
dcgan = DCGAN(hps)
z_placeholder, img_placeholder, generated_imgs, losses = dcgan.build()
train_op = dcgan.build_train(losses, hps.learning_rate, hps.beta1)

W0831 12:41:16.543827 140308269344576 deprecation.py:323] From <ipython-input-3-d1ca24847029>:43: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0831 12:41:16.545743 140308269344576 deprecation.py:506] From /root/anaconda3/envs/deep_learning/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0831 12:41:16.727539 140308269344576 deprecation.py:323] From <ipython-input-3-d1ca24847029>:45: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_depen

In [5]:
def combine_and_show_imgs(batch_imgs, img_size, rows=8, cols=16):
    """连接图片，组成一个网格图片"""
    # batch_imgs: [batch_size, img_size, img_size, 1]
    result_big_img = []
    for i in range(rows):
        row_imgs = []
        for j in range(cols):
            img = batch_imgs[cols * i + j]
            img = img.reshape((img_size, img_size))
            img = (img + 1) * 127.5
            row_imgs.append(img)
        row_imgs = np.hstack(row_imgs)
        result_big_img.append(row_imgs)
    result_big_img = np.vstack(result_big_img)
    result_big_img = np.asarray(result_big_img, np.uint8)
    result_big_img = Image.fromarray(result_big_img)
    return result_big_img


#开始训练
init_op = tf.global_variables_initializer()
train_steps = 10000
with tf.Session() as sess:
    sess.run(init_op)
    for step in range(train_steps):
        batch_img, batch_z = mnist_data.next_batch(hps.batch_size)
        
        fetches = [train_op, losses['g'], losses['d']]
        should_sample = (step + 1) % 50 == 0
        if should_sample:
            fetches += [generated_imgs]
        out_values = sess.run(fetches,
                              feed_dict = {
                                  z_placeholder: batch_z,
                                  img_placeholder: batch_img
                              })
        _, g_loss_val, d_loss_val = out_values[0:3]
        print('step: %d, g_loss: %4.3f, d_loss: %4.3f' % (step, g_loss_val, d_loss_val))
        if should_sample:
            gen_imgs_val = out_values[3]
            
            gen_img_path = os.path.join(output_dir, '%05d-gen.jpg' % (step + 1))
            gt_img_path = os.path.join(output_dir, '%05d-gt.jpg' % (step + 1))
            
            gen_img = combine_and_show_imgs(gen_imgs_val, hps.img_size)
            gt_img = combine_and_show_imgs(batch_img, hps.img_size)
             
            print(gen_img_path)
            print(gt_img_path)
            gen_img.save(gen_img_path)
            gt_img.save(gt_img_path)

step: 0, g_loss: 0.859, d_loss: 1.502
step: 1, g_loss: 11.047, d_loss: 0.025
step: 2, g_loss: 0.000, d_loss: 17.383
step: 3, g_loss: 25.452, d_loss: 6.703
step: 4, g_loss: 18.591, d_loss: 0.005
step: 5, g_loss: 1.433, d_loss: 0.422
step: 6, g_loss: 0.481, d_loss: 1.418
step: 7, g_loss: 10.698, d_loss: 0.093
step: 8, g_loss: 2.583, d_loss: 0.737
step: 9, g_loss: 10.377, d_loss: 2.209
step: 10, g_loss: 1.481, d_loss: 0.308
step: 11, g_loss: 8.354, d_loss: 0.001
step: 12, g_loss: 5.804, d_loss: 0.004
step: 13, g_loss: 0.002, d_loss: 6.941
step: 14, g_loss: 14.522, d_loss: 3.429
step: 15, g_loss: 13.181, d_loss: 0.206
step: 16, g_loss: 11.749, d_loss: 0.015
step: 17, g_loss: 10.620, d_loss: 0.007
step: 18, g_loss: 9.648, d_loss: 0.002
step: 19, g_loss: 8.581, d_loss: 0.001
step: 20, g_loss: 7.191, d_loss: 0.003
step: 21, g_loss: 4.934, d_loss: 0.009
step: 22, g_loss: 1.296, d_loss: 0.363
step: 23, g_loss: 2.491, d_loss: 0.111
step: 24, g_loss: 0.933, d_loss: 0.981
step: 25, g_loss: 12.619,

step: 202, g_loss: 4.659, d_loss: 0.073
step: 203, g_loss: 1.945, d_loss: 0.317
step: 204, g_loss: 1.500, d_loss: 0.346
step: 205, g_loss: 2.439, d_loss: 0.152
step: 206, g_loss: 1.598, d_loss: 0.314
step: 207, g_loss: 1.729, d_loss: 0.289
step: 208, g_loss: 3.519, d_loss: 0.197
step: 209, g_loss: 3.878, d_loss: 0.284
step: 210, g_loss: 2.307, d_loss: 0.160
step: 211, g_loss: 1.186, d_loss: 0.482
step: 212, g_loss: 3.751, d_loss: 0.326
step: 213, g_loss: 2.213, d_loss: 0.383
step: 214, g_loss: 1.106, d_loss: 0.693
step: 215, g_loss: 6.416, d_loss: 1.115
step: 216, g_loss: 2.349, d_loss: 0.294
step: 217, g_loss: 0.527, d_loss: 1.177
step: 218, g_loss: 8.311, d_loss: 1.402
step: 219, g_loss: 7.100, d_loss: 0.086
step: 220, g_loss: 5.814, d_loss: 0.023
step: 221, g_loss: 4.560, d_loss: 0.026
step: 222, g_loss: 3.683, d_loss: 0.060
step: 223, g_loss: 3.277, d_loss: 0.102
step: 224, g_loss: 3.471, d_loss: 0.056
step: 225, g_loss: 3.282, d_loss: 0.073
step: 226, g_loss: 2.562, d_loss: 0.170


step: 402, g_loss: 0.671, d_loss: 0.968
step: 403, g_loss: 4.829, d_loss: 1.076
step: 404, g_loss: 0.934, d_loss: 1.167
step: 405, g_loss: 3.022, d_loss: 0.615
step: 406, g_loss: 2.374, d_loss: 0.420
step: 407, g_loss: 1.654, d_loss: 0.430
step: 408, g_loss: 3.920, d_loss: 0.247
step: 409, g_loss: 2.491, d_loss: 0.265
step: 410, g_loss: 1.676, d_loss: 0.516
step: 411, g_loss: 2.421, d_loss: 0.449
step: 412, g_loss: 1.167, d_loss: 0.797
step: 413, g_loss: 4.342, d_loss: 1.652
step: 414, g_loss: 0.036, d_loss: 4.088
step: 415, g_loss: 5.543, d_loss: 1.064
step: 416, g_loss: 4.398, d_loss: 0.522
step: 417, g_loss: 2.357, d_loss: 0.290
step: 418, g_loss: 1.312, d_loss: 0.476
step: 419, g_loss: 2.310, d_loss: 0.238
step: 420, g_loss: 3.012, d_loss: 0.237
step: 421, g_loss: 2.845, d_loss: 0.265
step: 422, g_loss: 1.443, d_loss: 0.416
step: 423, g_loss: 2.329, d_loss: 0.382
step: 424, g_loss: 1.932, d_loss: 0.404
step: 425, g_loss: 1.970, d_loss: 0.369
step: 426, g_loss: 2.332, d_loss: 0.269


step: 602, g_loss: 3.326, d_loss: 0.353
step: 603, g_loss: 1.400, d_loss: 0.920
step: 604, g_loss: 0.861, d_loss: 1.126
step: 605, g_loss: 4.052, d_loss: 2.051
step: 606, g_loss: 0.416, d_loss: 1.586
step: 607, g_loss: 5.026, d_loss: 0.815
step: 608, g_loss: 4.094, d_loss: 0.203
step: 609, g_loss: 1.211, d_loss: 0.546
step: 610, g_loss: 1.963, d_loss: 0.455
step: 611, g_loss: 2.207, d_loss: 0.501
step: 612, g_loss: 0.610, d_loss: 1.438
step: 613, g_loss: 3.706, d_loss: 1.803
step: 614, g_loss: 0.167, d_loss: 2.555
step: 615, g_loss: 5.301, d_loss: 1.757
step: 616, g_loss: 3.861, d_loss: 0.580
step: 617, g_loss: 1.440, d_loss: 0.400
step: 618, g_loss: 1.445, d_loss: 0.369
step: 619, g_loss: 2.522, d_loss: 0.179
step: 620, g_loss: 2.733, d_loss: 0.242
step: 621, g_loss: 2.101, d_loss: 0.355
step: 622, g_loss: 2.204, d_loss: 0.379
step: 623, g_loss: 1.687, d_loss: 0.423
step: 624, g_loss: 1.436, d_loss: 0.570
step: 625, g_loss: 2.801, d_loss: 0.545
step: 626, g_loss: 1.088, d_loss: 0.734


step: 802, g_loss: 1.777, d_loss: 0.465
step: 803, g_loss: 1.155, d_loss: 0.723
step: 804, g_loss: 2.290, d_loss: 0.731
step: 805, g_loss: 0.714, d_loss: 1.047
step: 806, g_loss: 4.138, d_loss: 1.683
step: 807, g_loss: 0.191, d_loss: 2.119
step: 808, g_loss: 4.399, d_loss: 0.631
step: 809, g_loss: 3.912, d_loss: 0.878
step: 810, g_loss: 0.359, d_loss: 1.497
step: 811, g_loss: 2.253, d_loss: 0.610
step: 812, g_loss: 2.137, d_loss: 0.373
step: 813, g_loss: 0.862, d_loss: 1.171
step: 814, g_loss: 2.144, d_loss: 0.932
step: 815, g_loss: 0.612, d_loss: 1.083
step: 816, g_loss: 2.718, d_loss: 1.430
step: 817, g_loss: 0.979, d_loss: 0.864
step: 818, g_loss: 1.832, d_loss: 0.817
step: 819, g_loss: 1.225, d_loss: 0.883
step: 820, g_loss: 2.194, d_loss: 0.736
step: 821, g_loss: 1.791, d_loss: 0.421
step: 822, g_loss: 1.247, d_loss: 0.609
step: 823, g_loss: 2.805, d_loss: 0.723
step: 824, g_loss: 0.907, d_loss: 0.758
step: 825, g_loss: 2.359, d_loss: 0.655
step: 826, g_loss: 1.735, d_loss: 0.743


...

step: 9204, g_loss: 2.517, d_loss: 0.313
step: 9205, g_loss: 1.558, d_loss: 0.470
step: 9206, g_loss: 4.837, d_loss: 0.259
step: 9207, g_loss: 3.200, d_loss: 0.199
step: 9208, g_loss: 2.687, d_loss: 0.240
step: 9209, g_loss: 2.981, d_loss: 0.108
step: 9210, g_loss: 2.028, d_loss: 0.415
step: 9211, g_loss: 3.915, d_loss: 0.146
step: 9212, g_loss: 2.532, d_loss: 0.290
step: 9213, g_loss: 3.500, d_loss: 0.366
step: 9214, g_loss: 0.935, d_loss: 0.807
step: 9215, g_loss: 5.457, d_loss: 0.824
step: 9216, g_loss: 2.888, d_loss: 0.167
step: 9217, g_loss: 2.744, d_loss: 0.203
step: 9218, g_loss: 2.907, d_loss: 0.277
step: 9219, g_loss: 3.241, d_loss: 0.224
step: 9220, g_loss: 4.282, d_loss: 0.054
step: 9221, g_loss: 1.346, d_loss: 0.506
step: 9222, g_loss: 7.057, d_loss: 1.648
step: 9223, g_loss: 0.399, d_loss: 1.563
step: 9224, g_loss: 6.280, d_loss: 1.080
step: 9225, g_loss: 3.898, d_loss: 0.203
step: 9226, g_loss: 1.164, d_loss: 0.782
step: 9227, g_loss: 5.165, d_loss: 0.414
step: 9228, g_lo

step: 9400, g_loss: 3.827, d_loss: 0.070
step: 9401, g_loss: 5.353, d_loss: 0.146
step: 9402, g_loss: 2.235, d_loss: 0.203
step: 9403, g_loss: 5.517, d_loss: 0.056
step: 9404, g_loss: 5.147, d_loss: 0.091
step: 9405, g_loss: 2.844, d_loss: 0.122
step: 9406, g_loss: 3.271, d_loss: 0.122
step: 9407, g_loss: 4.537, d_loss: 0.145
step: 9408, g_loss: 4.227, d_loss: 0.055
step: 9409, g_loss: 4.509, d_loss: 0.064
step: 9410, g_loss: 3.721, d_loss: 0.062
step: 9411, g_loss: 3.046, d_loss: 0.162
step: 9412, g_loss: 3.542, d_loss: 0.058
step: 9413, g_loss: 4.832, d_loss: 0.042
step: 9414, g_loss: 5.651, d_loss: 0.059
step: 9415, g_loss: 2.576, d_loss: 0.284
step: 9416, g_loss: 4.645, d_loss: 0.054
step: 9417, g_loss: 4.404, d_loss: 0.069
step: 9418, g_loss: 4.031, d_loss: 0.082
step: 9419, g_loss: 3.350, d_loss: 0.078
step: 9420, g_loss: 4.835, d_loss: 0.067
step: 9421, g_loss: 4.680, d_loss: 0.034
step: 9422, g_loss: 4.113, d_loss: 0.064
step: 9423, g_loss: 3.827, d_loss: 0.071
step: 9424, g_lo

step: 9597, g_loss: 1.238, d_loss: 0.576
step: 9598, g_loss: 7.571, d_loss: 0.317
step: 9599, g_loss: 6.350, d_loss: 0.157
./local_run/09600-gen.jpg
./local_run/09600-gt.jpg
step: 9600, g_loss: 3.362, d_loss: 0.107
step: 9601, g_loss: 1.330, d_loss: 0.537
step: 9602, g_loss: 8.098, d_loss: 1.079
step: 9603, g_loss: 1.731, d_loss: 0.416
step: 9604, g_loss: 3.541, d_loss: 0.080
step: 9605, g_loss: 6.067, d_loss: 0.268
step: 9606, g_loss: 2.069, d_loss: 0.343
step: 9607, g_loss: 5.361, d_loss: 0.113
step: 9608, g_loss: 3.218, d_loss: 0.228
step: 9609, g_loss: 4.588, d_loss: 0.033
step: 9610, g_loss: 3.464, d_loss: 0.063
step: 9611, g_loss: 3.724, d_loss: 0.070
step: 9612, g_loss: 3.531, d_loss: 0.113
step: 9613, g_loss: 3.942, d_loss: 0.126
step: 9614, g_loss: 2.764, d_loss: 0.119
step: 9615, g_loss: 4.649, d_loss: 0.131
step: 9616, g_loss: 3.508, d_loss: 0.073
step: 9617, g_loss: 4.173, d_loss: 0.045
step: 9618, g_loss: 4.680, d_loss: 0.039
step: 9619, g_loss: 3.258, d_loss: 0.135
step: 

step: 9792, g_loss: 7.455, d_loss: 0.006
step: 9793, g_loss: 6.697, d_loss: 0.054
step: 9794, g_loss: 4.474, d_loss: 0.041
step: 9795, g_loss: 5.155, d_loss: 0.015
step: 9796, g_loss: 5.123, d_loss: 0.016
step: 9797, g_loss: 6.976, d_loss: 0.012
step: 9798, g_loss: 6.833, d_loss: 0.011
step: 9799, g_loss: 6.676, d_loss: 0.017
./local_run/09800-gen.jpg
./local_run/09800-gt.jpg
step: 9800, g_loss: 4.105, d_loss: 0.032
step: 9801, g_loss: 5.466, d_loss: 0.018
step: 9802, g_loss: 5.631, d_loss: 0.012
step: 9803, g_loss: 7.848, d_loss: 0.013
step: 9804, g_loss: 7.959, d_loss: 0.008
step: 9805, g_loss: 5.905, d_loss: 0.018
step: 9806, g_loss: 5.663, d_loss: 0.010
step: 9807, g_loss: 5.819, d_loss: 0.046
step: 9808, g_loss: 4.217, d_loss: 0.036
step: 9809, g_loss: 6.237, d_loss: 0.007
step: 9810, g_loss: 7.002, d_loss: 0.009
step: 9811, g_loss: 3.823, d_loss: 0.045
step: 9812, g_loss: 6.737, d_loss: 0.008
step: 9813, g_loss: 7.737, d_loss: 0.025
step: 9814, g_loss: 6.277, d_loss: 0.046
step: 

step: 9987, g_loss: 4.572, d_loss: 0.085
step: 9988, g_loss: 5.139, d_loss: 0.034
step: 9989, g_loss: 4.202, d_loss: 0.046
step: 9990, g_loss: 5.149, d_loss: 0.029
step: 9991, g_loss: 3.272, d_loss: 0.093
step: 9992, g_loss: 4.577, d_loss: 0.035
step: 9993, g_loss: 4.426, d_loss: 0.069
step: 9994, g_loss: 4.703, d_loss: 0.052
step: 9995, g_loss: 2.892, d_loss: 0.118
step: 9996, g_loss: 3.640, d_loss: 0.172
step: 9997, g_loss: 4.562, d_loss: 0.078
step: 9998, g_loss: 1.867, d_loss: 0.333
step: 9999, g_loss: 6.941, d_loss: 0.762
./local_run/10000-gen.jpg
./local_run/10000-gt.jpg
